In [15]:
from logging import WARNING, INFO
from typing import Callable, Dict, List, Optional, Tuple, Union
import numpy as np
import yaml

In [16]:
save_path = './conf/topologies/'

In [17]:
def save_to_yaml(max_num_clients_per_round: int, topology: List[List[int]], clients_with_no_data: List[int], last_connected_client: int,  path: str, name: str):
    num_nodes = len(topology)
    #GPU_share = 1.0/max_num_clients_per_round
    pools = {}
    for node in range(num_nodes):
        pools['p'+str(node)] = topology[node]
    data = {
        'num_clients': num_nodes,
        'max_num_clients_per_round': max_num_clients_per_round,
        'clients_with_no_data': clients_with_no_data,
        'last_connected_client': last_connected_client,
        'pools': pools
    }
    
    with open(path+name, 'w') as yaml_file:
        yaml.dump(data, yaml_file, default_flow_style=False, sort_keys=False)


In [18]:
def generate_chain(num_nodes: int, islands: Optional[int], leafs: Optional[int]=0):
    pool = []
    for i in range(num_nodes):
        if i == 0:
            pool.append([i, i+1])
        elif i == num_nodes-1:
            pool.append([i, i-1])
        else:
            pool.append([i, i-1, i+1])
    for i in range(num_nodes, num_nodes+islands):
        pool.append([i])
    return pool

In [19]:
chain = generate_chain(num_nodes=30,islands=5)
print(chain)
save_to_yaml(max_num_clients_per_round=3, topology=chain, clients_with_no_data=[0,5,10,33,34], last_connected_client=29, path=save_path, name='chain30+5.yaml')

[[0, 1], [1, 0, 2], [2, 1, 3], [3, 2, 4], [4, 3, 5], [5, 4, 6], [6, 5, 7], [7, 6, 8], [8, 7, 9], [9, 8, 10], [10, 9, 11], [11, 10, 12], [12, 11, 13], [13, 12, 14], [14, 13, 15], [15, 14, 16], [16, 15, 17], [17, 16, 18], [18, 17, 19], [19, 18, 20], [20, 19, 21], [21, 20, 22], [22, 21, 23], [23, 22, 24], [24, 23, 25], [25, 24, 26], [26, 25, 27], [27, 26, 28], [28, 27, 29], [29, 28], [30], [31], [32], [33], [34]]


In [41]:
def generate_cross_chain(num_crosses: int, islands: Optional[int]):
    pool = []
    num_crosses += 2 #First and last nodes in axis 0 are leafs
    for i in range(num_crosses): 
        if i == 0:
            pool.append([i, i+1])
        elif i == num_crosses-1:
            pool.append([i, i-1])
        else:
            pool.append([i, i-1, i+1])
    for i in range(0, num_crosses-2):
        pool[i+1] = np.concatenate((pool[i+1], [2*i + num_crosses, 2*i+ num_crosses+1]), axis=None).tolist()
        pool.append([2*i + num_crosses, i+1])
        pool.append([2*i + num_crosses + 1, i+1])
    
    for i in range(len(pool), len(pool) + islands):
        pool.append([i])
    return pool

In [42]:
cross_chain = generate_cross_chain(num_crosses=10, islands=5)
print(cross_chain)
save_to_yaml(max_num_clients_per_round=5, topology=cross_chain, clients_with_no_data=[0,5,15,20,35,36], last_connected_client=29, path=save_path, name='cross_chain30+5.yaml')

[[0, 1], [1, 0, 2, 12, 13], [2, 1, 3, 14, 15], [3, 2, 4, 16, 17], [4, 3, 5, 18, 19], [5, 4, 6, 20, 21], [6, 5, 7, 22, 23], [7, 6, 8, 24, 25], [8, 7, 9, 26, 27], [9, 8, 10, 28, 29], [10, 9, 11, 30, 31], [11, 10], [12, 1], [13, 1], [14, 2], [15, 2], [16, 3], [17, 3], [18, 4], [19, 4], [20, 5], [21, 5], [22, 6], [23, 6], [24, 7], [25, 7], [26, 8], [27, 8], [28, 9], [29, 9], [30, 10], [31, 10], [32], [33], [34], [35], [36]]


In [22]:
def generate_ring(num_nodes: int, islands: Optional[int], leafs: Optional[int]=0):
    pool = []
    for i in range(num_nodes):
        if i == 0:
            pool.append([i, num_nodes-1, i+1])
        elif i == num_nodes-1:
            pool.append([i, 0, i-1])
        else:
            pool.append([i, i-1, i+1])
    for i in range(num_nodes, num_nodes+islands):
        pool.append([i])
    return pool

In [23]:
ring = generate_ring(num_nodes=30, islands=5)
print(ring)
save_to_yaml(max_num_clients_per_round=3, topology=ring, clients_with_no_data=[0,5,10,33,34], last_connected_client=29, path=save_path, name='ring30+5.yaml')

[[0, 29, 1], [1, 0, 2], [2, 1, 3], [3, 2, 4], [4, 3, 5], [5, 4, 6], [6, 5, 7], [7, 6, 8], [8, 7, 9], [9, 8, 10], [10, 9, 11], [11, 10, 12], [12, 11, 13], [13, 12, 14], [14, 13, 15], [15, 14, 16], [16, 15, 17], [17, 16, 18], [18, 17, 19], [19, 18, 20], [20, 19, 21], [21, 20, 22], [22, 21, 23], [23, 22, 24], [24, 23, 25], [25, 24, 26], [26, 25, 27], [27, 26, 28], [28, 27, 29], [29, 0, 28], [30], [31], [32], [33], [34]]


In [24]:
def generate_ring_chain(num_rings: int, ring_size: int, islands: Optional[int], leafs: Optional[int]=0, joint_rings: Optional[bool] = False):
    pool = []
    for i in range(num_rings):
        for j in range(ring_size):
            if j == 0:
                pool.append([i*ring_size, i*ring_size+1, (i+1)*ring_size-1])
            elif j == ring_size - 1:
                pool.append([i*ring_size+j, i*ring_size+j-1, i*ring_size])
            else:
                pool.append([i*ring_size+j, i*ring_size+j-1, i*ring_size+j+1])
    for i in range(num_rings):
        if i == 0:
             pool[(i+1)*ring_size-1].append((i+1)*ring_size)
        elif i == num_rings-1:
             pool[(i*ring_size)].append(i*ring_size-1)
        else:
            pool[(i*ring_size)].append(i*ring_size-1)
            pool[(i+1)*ring_size-1].append((i+1)*ring_size)
    for i in range(num_rings*ring_size, num_rings*ring_size+islands):
        pool.append([i])
    return pool

In [25]:
ring_chain = generate_ring_chain(num_rings=6, ring_size=5, islands=5)
print(ring_chain)
save_to_yaml(max_num_clients_per_round=4, topology=ring_chain, clients_with_no_data=[0,5,10,33,34], last_connected_client=29, path=save_path, name='ring_chain30+5.yaml')

[[0, 1, 4], [1, 0, 2], [2, 1, 3], [3, 2, 4], [4, 3, 0, 5], [5, 6, 9, 4], [6, 5, 7], [7, 6, 8], [8, 7, 9], [9, 8, 5, 10], [10, 11, 14, 9], [11, 10, 12], [12, 11, 13], [13, 12, 14], [14, 13, 10, 15], [15, 16, 19, 14], [16, 15, 17], [17, 16, 18], [18, 17, 19], [19, 18, 15, 20], [20, 21, 24, 19], [21, 20, 22], [22, 21, 23], [23, 22, 24], [24, 23, 20, 25], [25, 26, 29, 24], [26, 25, 27], [27, 26, 28], [28, 27, 29], [29, 28, 25], [30], [31], [32], [33], [34]]


In [26]:
def generate_ring_ring(num_nodes: int, ring_sizes: int, islands: Optional[int], leafs: Optional[int]=0, joint_rings: Optional[bool] = False):
    None

In [27]:
def generate_star(num_nodes: int, islands: Optional[int], leafs: Optional[int]=0):
    None

In [28]:
# with open(save_path+'test.yaml', 'r') as file:
#     prime_service = yaml.safe_load(file)
# prime_service['pools']['p0']